# Research Analyzer - Multi-Analyst News Research System

This notebook implements a LangGraph-based research system that creates a team of specialized news analysts to conduct research on any topic and synthesize their findings into a comprehensive report.

## Phase 1: Setup & Environment

In this phase, we install dependencies and configure API keys.

In [ ]:
# Install required dependencies
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python python-dotenv pydantic

In [ ]:
import os
import getpass

def _set_env(var: str):
    """Set environment variable from user input if not already set."""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set OpenAI API Key
_set_env("OPENAI_API_KEY")

In [ ]:
# Set Tavily API Key for web search
_set_env("TAVILY_API_KEY")

In [ ]:
# Initialize the Language Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("✓ LLM initialized successfully!")

## Phase 2: Data Models

Define Pydantic models for NewsAnalyst, AnalystTeam, and SearchQuery.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class NewsAnalyst(BaseModel):
    """Model representing a specialized news analyst."""
    affiliation: str = Field(
        description="Primary affiliation of the analyst (e.g., News Network, Think Tank, Research Institute).",
    )
    name: str = Field(
        description="Name of the analyst."
    )
    role: str = Field(
        description="Role of the analyst (e.g., Political Correspondent, Tech Reporter, Economic Analyst).",
    )
    description: str = Field(
        description="Description of the analyst's focus, expertise, and analytical approach.",
    )
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

print("✓ NewsAnalyst model defined!")

In [ ]:
class AnalystTeam(BaseModel):
    """Model representing a team of news analysts."""
    analysts: List[NewsAnalyst] = Field(
        description="Team of news analysts with diverse specializations.",
    )

class SearchQuery(BaseModel):
    """Model for search query generation."""
    search_query: str = Field(None, description="Search query for news retrieval.")

print("✓ AnalystTeam and SearchQuery models defined!")

## Phase 3: State Definitions

Define TypedDict states for LangGraph workflows.

In [ ]:
import operator
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import MessagesState

class GenerateAnalystsState(TypedDict):
    """State for the analyst generation subgraph."""
    topic: str  # Topic to analyze
    max_analysts: int  # Number of analysts
    human_analyst_feedback: str  # Human feedback
    analysts: List[NewsAnalyst]  # Generated analysts

print("✓ GenerateAnalystsState defined!")

In [ ]:
class AnalysisState(MessagesState):
    """State for individual analyst research."""
    max_num_turns: int  # Number turns of conversation
    context: Annotated[list, operator.add]  # Source docs
    analyst: NewsAnalyst  # Analyst conducting analysis
    analysis: str  # Analysis transcript
    sections: Annotated[list, operator.add]  # Final sections for report

print("✓ AnalysisState defined!")

In [ ]:
class ResearchGraphState(TypedDict):
    """State for the main research graph."""
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[NewsAnalyst]
    sections: Annotated[list, operator.add]
    introduction: str
    content: str
    conclusion: str
    final_report: str

print("✓ ResearchGraphState defined!")